In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
document.title='AiiDAlab SSSP app'

In [ ]:
import ipywidgets as ipw
from jinja2 import Environment
from importlib_resources import files
from IPython.display import display

from aiida.orm import load_node
from aiidalab_widgets_base import WizardAppWidget, WizardAppWidgetStep
from aiidalab_widgets_base.bug_report import install_create_github_issue_exception_handler

from aiidalab_sssp import static
from aiidalab_sssp.process import WorkChainSelector
from aiidalab_sssp.steps import SubmitSsspWorkChainStep
from aiidalab_sssp.steps import ConfigureSsspWorkChainStep
from aiidalab_sssp.steps import ViewSsspAppWorkChainStatusAndResultsStep
from aiidalab_sssp.steps import PseudoSelectionStep
from aiidalab_sssp.steps import SettingPseudoMetadataStep
from aiidalab_sssp.steps import ViewSsspAppWorkChainStatusAndResultsStep
from aiidalab_sssp.version import __version__

pseudo_selection_step = PseudoSelectionStep(auto_advance=True)
configure_sssp_app_work_chain_step = ConfigureSsspWorkChainStep(auto_advance=True)
setting_pseudo_metadata_step = SettingPseudoMetadataStep(auto_advance=True)
submit_sssp_work_chain_step = SubmitSsspWorkChainStep(auto_advance=True)
view_sssp_app_work_chain_status_and_results_step = ViewSsspAppWorkChainStatusAndResultsStep()

# Link the application steps
ipw.dlink((pseudo_selection_step, 'state'), (configure_sssp_app_work_chain_step, 'previous_step_state'))
ipw.dlink((configure_sssp_app_work_chain_step, 'state'), (setting_pseudo_metadata_step, 'previous_step_state'))
ipw.dlink((setting_pseudo_metadata_step, 'state'), (submit_sssp_work_chain_step, 'previous_step_state'))

ipw.dlink((pseudo_selection_step, 'confirmed_pseudo'), (submit_sssp_work_chain_step, 'pseudo'))
ipw.dlink((pseudo_selection_step, 'confirmed_pseudo'), (setting_pseudo_metadata_step, 'pseudo'))
ipw.dlink((pseudo_selection_step, 'pseudo_filename'), (setting_pseudo_metadata_step.metadata_settings, 'pseudo_filename'))
ipw.dlink((configure_sssp_app_work_chain_step, 'workchain_settings'), (submit_sssp_work_chain_step, 'workchain_settings'))
ipw.dlink((setting_pseudo_metadata_step, 'metadata_settings'), (submit_sssp_work_chain_step, 'metadata_settings'))

ipw.dlink((submit_sssp_work_chain_step, 'process'), (view_sssp_app_work_chain_status_and_results_step, 'process'))

# Add the application steps to the application
app = WizardAppWidget(
    steps=[
        ('Select pseudo', pseudo_selection_step),
        ('Configure work chain', configure_sssp_app_work_chain_step),
        ('Pseudopotential metadata setting', setting_pseudo_metadata_step),
        ('Choose computational resources', submit_sssp_work_chain_step),
        ('Status & Results', view_sssp_app_work_chain_status_and_results_step),
    ])

# Reset all subsequent steps in case that a new pseudo is selected
def _observe_pseudo_selection(change):
    with pseudo_selection_step.hold_sync():
        if pseudo_selection_step.confirmed_pseudo is not None and \
                pseudo_selection_step.confirmed_pseudo != change['new']:
            app.reset()
            
pseudo_selection_step.observe(_observe_pseudo_selection, 'pseudo')

# Add process selection header
work_chain_selector = WorkChainSelector(layout=ipw.Layout(width='auto'))

def _observe_process_selection(change):
    if change['old'] == change['new']:
        return

    pk = change['new']
    if pk is None:
        app.reset()
        app.selected_index = 0
    else:
        process = load_node(pk)
        with pseudo_selection_step.hold_sync():
            app.selected_index = 4
            pseudo_selection_step.pseudo = process.inputs.pseudo
            pseudo_selection_step.confirmed_pseudo = process.inputs.pseudo
            configure_sssp_app_work_chain_step.state = WizardAppWidgetStep.State.SUCCESS
            submit_sssp_work_chain_step.process = process

            
work_chain_selector.observe(_observe_process_selection, 'value')
ipw.dlink((submit_sssp_work_chain_step, 'process'), (work_chain_selector, 'value'),
    transform=lambda node: None if node is None else node.pk)

env = Environment()

template = files(static).joinpath("welcome.jinja").read_text()
style = files(static).joinpath("style.css").read_text()
welcome_message = ipw.HTML(env.from_string(template).render(style=style))
footer = ipw.HTML(f'<p style="text-align:right;">Copyright (c) 2022 AiiDAlab team (EPFL)&#8195Version: {__version__}</p>')

app_with_work_chain_selector = ipw.VBox(children=[work_chain_selector, app])

output = ipw.Output()
install_create_github_issue_exception_handler(
    output,
    url='https://github.com/aiidalab/aiidalab-sssp/issues/new',
    labels=('bug', 'automated-report'))

with output:
    display(welcome_message, app_with_work_chain_selector, footer)
    
display(output)

[inspect result](./inspect.ipynb?pk=35858)